## Part 1 : Data Processing

### 1.1 Reading in Lyrics dataset

Data Source is an already scraped lyrics dataset from azlyrics.  
- __Source:__ https://www.kaggle.com/datasets/albertsuarez/azlyrics  
    - created using this code: https://github.com/AlbertSuarez/azlyrics-scraper/tree/master
- __Date:__ latest as of Jan 31, 2020.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [10]:
df= pd.read_csv("./azlyrics-scraper/azlyrics_lyrics_a.csv",quotechar='"',usecols=range(5),nrows=0)

for filename in os.listdir('azlyrics-scraper/'):
    print(filename)
    df_temp = pd.read_csv(os.path.join('azlyrics-scraper/', filename),quotechar='"',usecols=range(5))
    df = pd.concat([df,df_temp])
print ("Loaded!")

azlyrics_lyrics_19.csv
azlyrics_lyrics_g.csv
azlyrics_lyrics_p.csv
azlyrics_lyrics_q.csv
azlyrics_lyrics_f.csv
azlyrics_lyrics_d.csv
azlyrics_lyrics_s.csv
azlyrics_lyrics_r.csv
azlyrics_lyrics_e.csv
azlyrics_lyrics_v.csv
azlyrics_lyrics_a.csv
azlyrics_lyrics_w.csv
azlyrics_lyrics_u.csv
azlyrics_lyrics_b.csv
azlyrics_lyrics_c.csv
azlyrics_lyrics_t.csv
azlyrics_lyrics_y.csv
azlyrics_lyrics_n.csv
azlyrics_lyrics_o.csv
azlyrics_lyrics_x.csv
azlyrics_lyrics_z.csv
azlyrics_lyrics_m.csv
azlyrics_lyrics_l.csv
azlyrics_lyrics_h.csv
azlyrics_lyrics_i.csv
azlyrics_lyrics_k.csv
azlyrics_lyrics_j.csv
Loaded!


In [11]:
df

,ARTIST_NAME,ARTIST_URL,SONG_NAME,SONG_URL,LYRICS
0,03 greedo,https://www.azlyrics.com/19/03greedo.html,sweet lady,https://www.azlyrics.com/lyrics/03greedo/sweet...,"its only one, 03, i'm from grape street, where..."
1,03 greedo,https://www.azlyrics.com/19/03greedo.html,mafia business,https://www.azlyrics.com/lyrics/03greedo/mafia...,"you gonna make me put you in a suit and tie, s..."
2,03 greedo,https://www.azlyrics.com/19/03greedo.html,paranoid,https://www.azlyrics.com/lyrics/03greedo/paran...,"we could kill it, yeah, we could, we could, we..."
3,03 greedo,https://www.azlyrics.com/19/03greedo.html,never bend,https://www.azlyrics.com/lyrics/03greedo/never...,"yeah, you lil bitch ass niggas steady speaking..."
4,03 greedo,https://www.azlyrics.com/19/03greedo.html,prayer for my lost,https://www.azlyrics.com/lyrics/03greedo/praye...,"x loaded up bro, never see me in the church, p..."
...,...,...,...,...,...
11970,jory boy,https://www.azlyrics.com/j/joryboy.html,por que cambiar,https://www.azlyrics.com/lyrics/joryboy/porque...,", los evos jedis, no entiendo como, esto suced..."
11971,jory boy,https://www.azlyrics.com/j/joryboy.html,imposible amor,https://www.azlyrics.com/lyrics/joryboy/imposi...,"yo se que lo que tuvimos anoche te gusto, echa..."
11972,jory boy,https://www.azlyrics.com/j/joryboy.html,bonita,https://www.azlyrics.com/lyrics/joryboy/bonita...,", dice, matando la liga baby, jory boy, baby d..."
11973,jory boy,https://www.azlyrics.com/j/joryboy.html,no me condenes,https://www.azlyrics.com/lyrics/joryboy/nomeco...,", no se lo que te sucedio, quien te convirtio ..."


In [91]:
vc = df['ARTIST_NAME'].value_counts()
vc

e-40                501
barbra streisand    421
beach boys          257
johnny cash         255
iam                 253
                   ... 
cory gunz             1
petite meller         1
cashmere cat          1
graceffa, joey        1
matt stell            1
Name: ARTIST_NAME, Length: 6464, dtype: int64

In [92]:
vc = vc[vc > 100]
vc = pd.DataFrame(vc).reset_index()
vc =vc.rename(columns={"ARTIST_NAME": "count", "index": "ARTIST_NAME"})
vc

,ARTIST_NAME,count
0,e-40,501
1,barbra streisand,421
2,beach boys,257
3,johnny cash,255
4,iam,253
...,...,...
124,jeremih,102
125,shirley bassey,102
126,"cash, johnny",101
127,garth brooks,101


### 1.2 Reading in wikipedia data on artists

In [155]:
# For web scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

In [32]:
#Single URL test
URL = 'https://en.wikipedia.org/w/api.php?action=opensearch&search=adrianocelentano&limit=1&namespace=0&format=json'
r = requests.get(URL)
r
r.json()[3]

b'["adrianocelentano",["Adriano Celentano"],[""],["https://en.wikipedia.org/wiki/Adriano_Celentano"]]'

In [93]:
#Creating wikipedia search URLs
vc['Artist_search_URL'] = 'https://en.wikipedia.org/w/api.php?action=opensearch&search=' +vc['ARTIST_NAME'].astype(str)+ '&limit=1&namespace=0&format=json'

In [121]:
#Time this takes : 21 seconds

#Declaring empty column
vc['Artist_wiki_URL'] = None

#Storing actual wikipeadia page URLs
for i in range(0,len(vc)):
    r = requests.get(vc.loc[i,'Artist_search_URL'])
    if len(r.json()[3]) > 0:
        vc.loc[i,'Artist_wiki_URL'] = r.json()[3][0]
    else:
        pass

vc
    

,ARTIST_NAME,count,Artist_search_URL,Artist_wiki_URL
0,e-40,501,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/E-40
1,barbra streisand,421,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Barbra_Streisand
2,beach boys,257,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Beach_boys
3,johnny cash,255,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Johnny_Cash
4,iam,253,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Iam
...,...,...,...,...
124,jeremih,102,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Jeremih_discography
125,shirley bassey,102,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Shirley_Bassey
126,"cash, johnny",101,https://en.wikipedia.org/w/api.php?action=open...,"https://en.wikipedia.org/wiki/Cash,_Johnny"
127,garth brooks,101,https://en.wikipedia.org/w/api.php?action=open...,https://en.wikipedia.org/wiki/Garth_Brooks


In [124]:
vc.loc[0,'Artist_wiki_URL']

'https://en.wikipedia.org/wiki/E-40'

In [214]:
# Send a get request and assign the response to a variable
wiki_response = requests.get(vc.loc[0,'Artist_wiki_URL'])

# Turn the undecoded content into a Beautiful Soup object and assign it to a variable
wiki_soup = BeautifulSoup(wiki_response.content)

In [239]:
data = []
table_body = wiki_soup.find('table', class_='infobox vcard plainlist')

rows = table_body.find_all('tr')
for row in rows:
    head =row.find_all('th')
    head = [ele.text.strip() for ele in head]
    print("head:",head)
    cols = row.find_all('td')
    print("before:  " ,cols)
    cols = [ele.text.strip() for ele in cols]
    print("After: " ,cols)
    data.append([head,cols]) # Get rid of empty values

head: ['E-40']
before:   []
After:  []
head: []
before:   [<td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:E-40_new.jpg"><img alt="E-40 new.jpg" data-file-height="612" data-file-width="424" decoding="async" height="318" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/E-40_new.jpg/220px-E-40_new.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/E-40_new.jpg/330px-E-40_new.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/a/aa/E-40_new.jpg 2x" width="220"/></a></td>]
After:  ['']
head: ['Background information']
before:   []
After:  []
head: ['Born']
before:   [<td class="infobox-data"><span style="display:none"> (<span class="bday">1967-11-15</span>) </span>November 15, 1967<span class="noprint ForceAgeToShow"> (age 55)</span><br/><a href="/wiki/Vallejo,_California" title="Vallejo, California">Vallejo, California</a>, U.S</td>]
After:  ['(1967-11-15) November 15, 1967 (age\xa055)Vallejo, California, U.S']
head: ['Origin']
before:   [<td c

In [240]:
pd.DataFrame(data)

,0,1
0,[E-40],[]
1,[],[]
2,[Background information],[]
3,[Born],"[(1967-11-15) November 15, 1967 (age 55)Vallej..."
4,[Origin],"[East Bay, California, U.S.]"
5,[Education],[Grambling State University (no degree)]
6,[],[]
7,[Genres],[Hip hop\nhyphy]
8,[Occupation(s)],[Rapper\nsongwriter\nentrepreneur\nactor\nMayo...
9,[Instrument(s)],[Rap vocals]


In [203]:
html_doc = requests.get(vc.loc[0,'Artist_wiki_URL']).content

tables = pd.read_html(html_doc)

print(f"{len(tables)} tables were found in this html content.")
index_gen = tables[0][tables[0][tables[0].columns[0]]=='Genres'].index

tables[0].iloc[index_gen[0],1]

4 tables were found in this html content.


'.mw-parser-output .hlist dl,.mw-parser-output .hlist ol,.mw-parser-output .hlist ul{margin:0;padding:0}.mw-parser-output .hlist dd,.mw-parser-output .hlist dt,.mw-parser-output .hlist li{margin:0;display:inline}.mw-parser-output .hlist.inline,.mw-parser-output .hlist.inline dl,.mw-parser-output .hlist.inline ol,.mw-parser-output .hlist.inline ul,.mw-parser-output .hlist dl dl,.mw-parser-output .hlist dl ol,.mw-parser-output .hlist dl ul,.mw-parser-output .hlist ol dl,.mw-parser-output .hlist ol ol,.mw-parser-output .hlist ol ul,.mw-parser-output .hlist ul dl,.mw-parser-output .hlist ul ol,.mw-parser-output .hlist ul ul{display:inline}.mw-parser-output .hlist .mw-empty-li{display:none}.mw-parser-output .hlist dt::after{content:": "}.mw-parser-output .hlist dd::after,.mw-parser-output .hlist li::after{content:" · ";font-weight:bold}.mw-parser-output .hlist dd:last-child::after,.mw-parser-output .hlist dt:last-child::after,.mw-parser-output .hlist li:last-child::after{content:none}.mw-pa

### 1.3 Transforming Lyrics via NLP

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer